In [3]:
#%pip install nbimporter
import find_eoms.ipynb

ModuleNotFoundError: No module named 'find_eoms'

In [2]:
import os
print(os.getcwd())  # Prints the current working directory
print(os.listdir())  # Prints all files in the current directory

/home/kian/Documents/Thesis/ICS_fork/ics-pa-sv/Kian_code
['outputs', '__init__.py', '.ipynb_checkpoints', 'Learn Transform.ipynb', 'find_eoms.ipynb', 'Find_A_theta.ipynb', 'Find_A_theta-Cosimo.ipynb']
